# Alpaca-py crypto trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/crypto-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with crypto trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for crypto as an example.

In [1]:
# Please change the following to your own PAPER api key and secret
# You can get them from https://alpaca.markets/

api_key = ""
secret_key = ""

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [2]:
# install alpaca-py
! python3 -m pip install alpaca-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 7.3 MB/s eta 0:00:00


In [3]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.crypto import CryptoHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.crypto import CryptoDataStream

from alpaca.data.requests import (
    CryptoBarsRequest,
    CryptoQuoteRequest,
    CryptoTradesRequest,
    CryptoLatestQuoteRequest
    )
from alpaca.trading.requests import (
    GetAssetsRequest,
    MarketOrderRequest,
    LimitOrderRequest,
    StopLimitOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest
)
from alpaca.trading.enums import (
    AssetClass,
    AssetStatus,
    OrderSide,
    OrderType,
    TimeInForce,
    QueryOrderStatus
)
from alpaca.common.exceptions import APIError

In [4]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [5]:
# check version of alpaca-py
alpaca.__version__

'0.34.0'

# Trading Client

In [6]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [7]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA3UQDPSHTQR',
    'accrued_fees': '0',
    'buying_power': '200000',
    'cash': '100000',
    'created_at': datetime.datetime(2024, 12, 10, 20, 8, 23, 440087, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '100000',
    'id': UUID('e8a1221d-e393-47f0-a9d8-8c7e892cfa72'),
    'initial_margin': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '100000',
    'options_approved_level': 2,
    'options_buying_power': '100000',
    'options_trading_level': 2,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '100000',
    'regt_buying_power': '200000',
    'short_market_value': '0',
    'shorting_enabled':

In [8]:
# check account configuration
# ref. https://docs.alpaca.markets/reference/getaccountconfig-1
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [9]:
# get list of crypto pairs
# ref. https://docs.alpaca.markets/reference/get-v2-assets-1
req = GetAssetsRequest(
  asset_class=AssetClass.CRYPTO,
  status=AssetStatus.ACTIVE
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
     'fractionable': True,
     'id': UUID('0515189d-1933-4a94-89ce-4e9a24356d58'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': 0.022640449,
     'min_trade_increment': 1e-09,
     'name': 'Avalanche / US Dollar',
     'price_increment': 0.0005,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'AVAX/USD',
     'tradable': True},
 {   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
     'fractionable': True,
     'id': UUID('90a83e2d-e574-404b-9344-74130736b71c'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': 0.000532708,
     'min_trade_increment': 1e-09,
     'name': 'Maker / US Dollar',
     'pri

### Orders

In [10]:
# we will place orders which Alapca trading platform supports
# - order types for crypto: market, limit, stop_limit
# - time_in_force for crypto: gtc, ioc.
# please refer to the following documents for more details
# ref. https://docs.alpaca.markets/docs/orders-at-alpaca
# ref. https://docs.alpaca.markets/reference/postorder


# we will place orders for symbol: BTC/USD in this example
symbol = "BTC/USD"

In [11]:
# simple, market order
# you can specify:
# or notional value (e.g. 100 USD) (which is in the next example)
req = MarketOrderRequest(
    symbol = symbol,
    qty = 0.01,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '70ef58f9-80da-460c-8997-9636462bd8f7',
    'created_at': datetime.datetime(2024, 12, 10, 21, 8, 7, 521771, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('e0e08a32-cdb0-414f-880d-fc5c6f2eb47d'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'qty': '0.01',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 12, 10, 21, 8, 7, 521771, tzinfo=TzInfo(UTC)),
    'symbol': 'BTC/USD',


In [12]:
# simple, market order, notional

req = MarketOrderRequest(
    symbol = symbol,
    notional = 1.11,  # notional is specified in USD, here we specify $1.11
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': 'c2e97b01-3525-49da-9f5c-57672bad0b07',
    'created_at': datetime.datetime(2024, 12, 10, 21, 9, 24, 353091, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('ef94afc8-0828-4dce-b06c-42d926e83444'),
    'legs': None,
    'limit_price': None,
    'notional': '1.11',
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 12, 10, 21, 9, 24, 353091, tzinfo=TzInfo(UTC)),
    'symbol': 'BTC/USD'

In [13]:
# simple, limit order
req = LimitOrderRequest(
    symbol = symbol,
    qty = 0.01,
    limit_price = 60000,
    side = OrderSide.BUY,
    type = OrderType.LIMIT,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '6fc250aa-5f61-4a4b-b90d-790affd7e70c',
    'created_at': datetime.datetime(2024, 12, 10, 21, 9, 30, 692444, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('5339a138-1d3c-48a0-b83e-5786ce788ffe'),
    'legs': None,
    'limit_price': '60000',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'qty': '0.01',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 12, 10, 21, 9, 30, 692444, tzinfo=TzInfo(UTC)),
    'symbol': 'BTC/USD

In [14]:
# stop limit order
req = StopLimitOrderRequest(
                    symbol = symbol,
                    qty = 0.01,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    limit_price = 55000,
                    stop_price = 60000
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '23d617a9-3dae-4889-aef7-b2f7f78b0ddd',
    'created_at': datetime.datetime(2024, 12, 10, 21, 9, 39, 277945, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('207dbfed-7883-4e36-afa9-2137acc26246'),
    'legs': None,
    'limit_price': '55000',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
    'qty': '0.01',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.NEW: 'new'>,
    'stop_price': '60000',
    'submitted_at': datetime.datetime(2024, 12, 10, 21, 9, 39, 277945, tzinfo=TzInfo(UTC)),
    'symbol': 'BTC/USD',


In [15]:
# get a list of orders including closed (e.g. filled) orders by specifying symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [symbol]
)
orders = trade_client.get_orders(req)
orders

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
     'canceled_at': None,
     'client_order_id': '23d617a9-3dae-4889-aef7-b2f7f78b0ddd',
     'created_at': datetime.datetime(2024, 12, 10, 21, 9, 39, 277945, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('207dbfed-7883-4e36-afa9-2137acc26246'),
     'legs': None,
     'limit_price': '55000',
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
     'qty': '0.01',
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.NEW: 'new'>,
     'stop_price': '60000',
     'submitted_at': datetime.datetime(2024, 12, 10, 21, 9, 39, 277945, tzinfo=TzInfo(UTC)),

In [16]:
# see all open orders
req = GetOrdersRequest(
    status = QueryOrderStatus.OPEN,
    symbols = [symbol]
)
open_orders = trade_client.get_orders(req)
open_orders

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
     'canceled_at': None,
     'client_order_id': '23d617a9-3dae-4889-aef7-b2f7f78b0ddd',
     'created_at': datetime.datetime(2024, 12, 10, 21, 9, 39, 277945, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('207dbfed-7883-4e36-afa9-2137acc26246'),
     'legs': None,
     'limit_price': '55000',
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
     'qty': '0.01',
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.NEW: 'new'>,
     'stop_price': '60000',
     'submitted_at': datetime.datetime(2024, 12, 10, 21, 9, 39, 277945, tzinfo=TzInfo(UTC)),

In [17]:
# cancel all open orders
trade_client.cancel_orders()

[{   'body': None,
     'id': UUID('207dbfed-7883-4e36-afa9-2137acc26246'),
     'status': 200},
 {   'body': None,
     'id': UUID('5339a138-1d3c-48a0-b83e-5786ce788ffe'),
     'status': 200}]

### Positions

In [18]:
# get all open positions
# ref. https://docs.alpaca.markets/reference/getallopenpositions-1
positions = trade_client.get_all_positions()
positions

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'asset_id': UUID('64bbff51-59d6-4b3c-9351-13ad85e3c752'),
     'asset_marginable': False,
     'avg_entry_price': '96820.132512969',
     'avg_entry_swap_rate': None,
     'change_today': '-0.0066384878028714',
     'cost_basis': '966.866079',
     'current_price': '96590.5',
     'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
     'lastday_price': '97236',
     'market_value': '964.57292',
     'qty': '0.009986209',
     'qty_available': '0.009986209',
     'side': <PositionSide.LONG: 'long'>,
     'swap_rate': None,
     'symbol': 'BTCUSD',
     'unrealized_intraday_pl': '-2.293159',
     'unrealized_intraday_plpc': '-0.0023717441844394',
     'unrealized_pl': '-2.293159',
     'unrealized_plpc': '-0.0023717441844394',
     'usd': None}]

In [19]:
# get positions by symbol
# ref. https://docs.alpaca.markets/reference/getopenposition-1
position = trade_client.get_open_position(symbol_or_asset_id=symbol)
position


APIError: Not Found

In [ ]:
# get positions by asset_id
trade_client.get_open_position(symbol_or_asset_id=position.asset_id)

In [ ]:
# close the position with specifying qty
# ref. https://docs.alpaca.markets/reference/deleteopenposition-1
trade_client.close_position(
    symbol_or_asset_id = symbol,
    close_options = ClosePositionRequest(
        qty = "0.01",
    )
)

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

You can open this notebook in another window and run below cell to check trade updates.

In [ ]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

# Market Data (Historical)

In [ ]:
# setup crypto historical data client
crypto_historical_data_client = CryptoHistoricalDataClient()

In [ ]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/cryptobars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = CryptoBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 1),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
crypto_historical_data_client.get_crypto_bars(req).df

In [ ]:
# get historical quote by symbol
# ref. https://docs.alpaca.markets/reference/cryptoquotes-1
now = datetime.now(ZoneInfo("America/New_York"))
req = CryptoQuoteRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 1),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
crypto_historical_data_client.get_crypto_quotes(req).df

In [ ]:
# get historical trades by symbol
req = CryptoTradesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 1),                          # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                                # specify limit
)
crypto_historical_data_client.get_crypto_trades(req).df

In [ ]:
# get latest quotes by symbol
req = CryptoLatestQuoteRequest(
    symbol_or_symbols = [symbol],
)
res = crypto_historical_data_client.get_crypto_latest_quote(req)
res